# UF NaviGator API Testing with Mistral-Small-3.1
## Educational AI Language Model Integration

Welcome to the third notebook in our AI series! This notebook explores how to use University of Florida's NaviGator API to access state-of-the-art language models like Mistral-Small-3.1.

### 🎯 Learning Objectives
By the end of this notebook, you will understand:
- How to authenticate and connect to UF's NaviGator API
- How to use OpenAI-compatible APIs for language models
- How to craft effective prompts for different tasks
- How to integrate language models into game applications
- Best practices for API usage and error handling

### 🚀 What is NaviGator?
NaviGator is University of Florida's AI platform that provides access to powerful language models for research and educational purposes. It uses an OpenAI-compatible API, making it easy to integrate with existing tools and workflows.

### 🎮 Game Integration Potential
In our farming game, we can use language models for:
- **Dynamic NPC Dialogue**: Generate contextual responses based on player actions
- **Quest Generation**: Create unique missions and storylines
- **Educational Content**: Provide farming tips and explanations
- **Emotion-Aware Responses**: Adapt dialogue based on detected player emotions

## 📦 Required Libraries and Setup

First, let's install and import the necessary libraries for API communication.

In [ ]:
# Install required packages
import subprocess
import sys


def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Successfully installed {package}")
    except subprocess.CalledProcessError:
        print(f"❌ Failed to install {package}")


# Install required packages
packages = ["openai", "requests"]
for package in packages:
    install_package(package)

# Import required libraries
import openai
from openai import OpenAI
import os
import json
import requests
from typing import Dict, List, Optional
import time

print("📚 Libraries imported successfully!")

## 🔐 API Authentication and Configuration

For this notebook to work, you'll need to create a JSON file with your NaviGator API credentials. The file should contain your API key and base URL.

**Note**: In a real implementation, you would store these credentials securely and never commit them to version control.

In [ ]:
# Configuration for NaviGator API
# We'll use the navigator_api_key.json file in the ai_materials folder


def load_api_credentials(key_file_path: str) -> Dict[str, str]:
    """
    Load API credentials from JSON file

    Args:
        key_file_path: Path to the JSON file containing API credentials

    Returns:
        Dictionary containing API key and base URL
    """
    try:
        with open(key_file_path, "r") as file:
            data = json.load(file)

        # Extract the required values
        api_key = data.get("OPENAI_API_KEY")
        base_url = data.get("base_url")

        if not api_key or not base_url:
            raise ValueError("Missing required credentials in JSON file")

        return {"api_key": api_key, "base_url": base_url}
    except FileNotFoundError:
        print("❌ Credentials file not found!")
        print("📝 Please create a JSON file with the following structure:")
        print(
            """
{
    "OPENAI_API_KEY": "your-api-key-here",
    "base_url": "your-base-url-here"
}
            """
        )
        return None
    except json.JSONDecodeError:
        print("❌ Invalid JSON format in credentials file")
        return None
    except Exception as e:
        print(f"❌ Error loading credentials: {e}")
        return None


# Load credentials from the actual file
key_file = "navigator_api_key.json"

print("🔧 Loading API credentials...")
credentials = load_api_credentials(key_file)

if credentials:
    print("✅ Credentials loaded successfully!")
else:
    print("❌ Failed to load credentials")
    credentials = None

## 🤖 Setting Up the OpenAI Client

Once you have your credentials, here's how to set up the OpenAI client to work with NaviGator:

In [ ]:
def setup_navigator_client(credentials: Dict[str, str]) -> openai.OpenAI:
    """
    Set up OpenAI client for NaviGator API

    Args:
        credentials: Dictionary containing API key and base URL

    Returns:
        Configured OpenAI client
    """
    # Set environment variable for the API key
    os.environ["TOOLKIT_API_KEY"] = credentials["api_key"]

    # Initialize OpenAI client with UF NaviGator API
    if credentials:
        client = OpenAI(
            api_key=credentials["api_key"], base_url=credentials["base_url"]
        )
        print("✅ OpenAI client initialized with UF NaviGator API")
        print(f"🌐 Connected to: {credentials['base_url']}")
        print("📊 Ready to test API endpoints!")
    else:
        print("❌ Cannot initialize client without credentials")
        print("Please ensure navigator_api_key.json is properly configured")
        client = None

    return client


# Example setup (you would uncomment and use this with real credentials)
# client = setup_navigator_client(credentials)
print("🔧 Navigator client setup function defined")

In [ ]:
# Actually initialize the client with our loaded credentials
if credentials:
    client = OpenAI(api_key=credentials["api_key"], base_url=credentials["base_url"])
    print("✅ OpenAI client initialized with UF NaviGator API")
    print(f"🌐 Connected to: {credentials['base_url']}")
    print("📊 Ready to test API endpoints!")
else:
    print("❌ Cannot initialize client without credentials")
    client = None

## 📋 Exploring Available Models

Let's check what models are available through the NaviGator API:

In [ ]:
def list_available_models(client: openai.OpenAI) -> List[str]:
    """
    Get list of available models from the API

    Args:
        client: Configured OpenAI client

    Returns:
        List of available model names
    """
    try:
        response = client.models.list()
        models = [model.id for model in response.data]

        print("🤖 Available Models:")
        print("=" * 50)
        for i, model in enumerate(models, 1):
            print(f"{i:2d}. {model}")

        return models
    except Exception as e:
        print(f"❌ Error fetching models: {e}")
        return []


# List available models using the client
list_available_models(client)

## 💬 Basic Chat Completion

Now let's test basic chat completion with the LLama-3.1-8b-instruct model:

In [ ]:
def test_basic_chat(client: openai.OpenAI, model: str = "llama-3.1-8b-instruct") -> str:
    """
    Test basic chat completion with the specified model

    Args:
        client: Configured OpenAI client
        model: Model name to use for completion

    Returns:
        Generated response text
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": "Write a short poem about something fun."}
            ],
            max_tokens=150,  # Limit response length
            temperature=0.7,  # Control randomness of output
        )

        # Extract the response
        generated_text = response.choices[0].message.content

        print(f"🤖 Model: {model}")
        print("📝 Generated Response:")
        print("-" * 50)
        print(generated_text)
        print("-" * 50)

        return generated_text

    except Exception as e:
        print(f"❌ Error generating response: {e}")
        return ""


# Example usage (commented out - requires real API credentials)
test_basic_chat(client, "llama-3.1-8b-instruct")

## 🎮 Game-Specific AI Integration

Let's explore how we can use language models specifically for our farming game. We'll create functions that generate content relevant to our PyDew Valley game:

In [ ]:
def generate_npc_dialogue(
    client: openai.OpenAI,
    character_name: str,
    character_role: str,
    emotion: str = "neutral",
) -> str:
    """





    Generate contextual NPC dialogue for the farming game






    Args:





        client: Configured OpenAI client





        character_name: Name of the NPC





        character_role: Role/profession of the NPC (e.g., "merchant", "farmer")





        emotion: Player's detected emotion






    Returns:





        Generated dialogue text
    """


    prompt = f"""





You are {character_name}, a {character_role} in a cozy farming game called PyDew Valley. 





Player's current emotion: {emotion}






Generate a short, friendly dialogue response (2-3 sentences) that:





1. Matches your character's role and personality





2. Responds appropriately to the player's context





3. Considers the player's emotional state





4. Maintains the game's wholesome, educational tone






Dialogue:
"""


    try:

        response = client.chat.completions.create(

            model="llama-3.1-8b-instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful NPC in a farming simulation game. Keep responses brief, friendly, and appropriate for all ages.",
                },
                {"role": "user", "content": prompt},
            ],

            max_tokens=100,
            temperature=0.8,
        )


        return response.choices[0].message.content.strip()


    except Exception as e:

        # Fallback dialogue if API fails

        return f"Hello there! Nice to see you around the farm today!"



# Example dialogue generation scenarios


dialogue_scenarios = [
    {
        "character_name": "Merchant Pete",
        "character_role": "trader",
        "emotion": "happy",
    },
    {
        "character_name": "Farmer Sarah",
        "character_role": "experienced farmer",
        "emotion": "sad",
    },
    {
        "character_name": "Blacksmith Tom",
        "character_role": "tool maker",
        "emotion": "angry",
    },
]



print("🎭 NPC Dialogue Generation System Ready!")


print("\n📝 Example Scenarios:")


for i, scenario in enumerate(dialogue_scenarios, 1):

    print(f"{i}. {scenario['character_name']} ({scenario['character_role']})")

    print(f"   Emotion: {scenario['emotion']}")

In [ ]:
# Test game-specific NPC dialogue generation
if client:
    try:
        print("🎭 Testing NPC dialogue generation for PyDew Valley...")

        # Test dialogue for Merchant Pete
        prompt = """
You are Merchant Pete, a friendly trader in a cozy farming game called PyDew Valley. 

Player context: The player just started farming and has very few coins but seems excited about learning
Player's current emotion: happy

Generate a short, friendly dialogue response (2-3 sentences) that:
1. Matches your character as a helpful trader
2. Responds to their beginner status and low funds
3. Acknowledges their positive attitude
4. Maintains the game's wholesome, educational tone

Dialogue:
"""

        response = client.chat.completions.create(
            model="llama-3.1-8b-instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful NPC in a farming simulation game. Keep responses brief, friendly, and appropriate for all ages.",
                },
                {"role": "user", "content": prompt},
            ],
            max_tokens=100,
            temperature=0.8,
        )

        dialogue = response.choices[0].message.content.strip()

        print("🎯 Generated NPC Dialogue:")
        print("=" * 50)
        print(f"Merchant Pete: {dialogue}")
        print("=" * 50)
        print("✅ Game dialogue generation successful!")

    except Exception as e:
        print(f"❌ Error generating dialogue: {e}")
        # Fallback dialogue
        print("🔄 Using fallback dialogue:")
        print(
            "Merchant Pete: Welcome to my shop, friend! I can see you're eager to start your farming journey. Don't worry about not having many coins yet - I've got some great starter deals for ambitious farmers like you!"
        )

else:
    print("❌ Client not initialized. Please run the client initialization cell first.")

## 🌱 Farming Game Content Generation

We could potentially use the language model to generate farming-related content, such as:

In [ ]:
def generate_farming_tips(
    client: openai.OpenAI, crop_type: str, season: str, skill_level: str = "beginner"
) -> str:
    """





    Generate educational farming tips for the game






    Args:





        client: Configured OpenAI client





        crop_type: Type of crop (e.g., "tomatoes", "corn", "carrots")





        season: Current season in game





        skill_level: Player's farming skill level






    Returns:





        Educational farming tip
    """


    prompt = f"""





Generate a helpful, educational farming tip for a {skill_level} player growing {crop_type} during {season}.






Make it:





- Scientifically accurate but simple to understand





- Relevant to the crop and season





- Encouraging and positive





- 1-2 sentences long






Tip:
"""


    try:

        response = client.chat.completions.create(

            model="mistral-small-3.1",
            messages=[
                {
                    "role": "system",
                    "content": "You are a knowledgeable farming educator. Provide helpful, accurate, and encouraging farming advice.",
                },
                {"role": "user", "content": prompt},
            ],
            max_tokens=80,

            temperature=0.6,
        )


        return response.choices[0].message.content.strip()


    except Exception as e:

        # Fallback tip if API fails

        return f"Remember to water your {crop_type} regularly and give them plenty of sunlight!"



# Example farming tip scenarios


farming_scenarios = [
    {"crop_type": "tomatoes", "season": "summer", "skill_level": "beginner"},
    {"crop_type": "corn", "season": "spring", "skill_level": "intermediate"},

    {"crop_type": "pumpkins", "season": "fall", "skill_level": "advanced"},
]



print("🌱 Farming Tips Generation System Ready!")


print("\n📚 Example Tip Scenarios:")



for scenario in farming_scenarios:

    print(
        f"• {scenario['crop_type'].title()} in {scenario['season']} ({scenario['skill_level']})"
    )

# Test farming tips generation
if client:
    try:
        print("🌱 Testing farming tips generation...")

        for scenario in farming_scenarios:
            tip = generate_farming_tips(
                client,
                crop_type=scenario["crop_type"],
                season=scenario["season"],
                skill_level=scenario["skill_level"],
            )
            print(
                f"• {scenario['crop_type'].title()} in {scenario['season']} ({scenario['skill_level']}): {tip}"
            )

        print("✅ Farming tips generation successful!")

    except Exception as e:
        print(f"❌ Error generating farming tips: {e}")

## 🔄 Emotion-Aware Dialogue System

One of the most exciting possibilities is combining our emotion detection with AI-generated dialogue. Let's create a system that adapts NPC responses based on the player's detected emotions:

In [ ]:
def create_emotion_aware_dialogue(
    client: openai.OpenAI,
    npc_name: str,
    detected_emotions: List[str],
) -> str:
    """







    Generate dialogue that responds to the player's emotional state








    Args:







        client: Configured OpenAI client







        npc_name: Name of the NPC








        detected_emotions: List of recently detected emotions (most recent first)








    Returns:







        Emotion-aware dialogue response
    """


    # Determine primary emotion (most recent) and emotional trend

    primary_emotion = detected_emotions[0] if detected_emotions else "neutral"

    emotion_trend = "consistent" if len(set(detected_emotions[:3])) <= 1 else "changing"


    prompt = f"""







You are {npc_name} in a farming game.








Player's emotional state:







- Current emotion: {primary_emotion}







- Recent emotions: {', '.join(detected_emotions[:3])}







- Emotional trend: {emotion_trend}








Generate a response that:







1. Acknowledges their emotional state appropriately







2. Provides comfort if they seem upset







3. Shares in their joy if they seem happy







4. Offers encouragement if they seem frustrated







5. Keeps the tone warm and supportive








Response (2-3 sentences):
"""

    try:


        response = client.chat.completions.create(

            model="llama-3.1-8b-instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are an empathetic NPC who cares about the player's wellbeing. Respond with emotional intelligence and kindness.",
                },
                {"role": "user", "content": prompt},
            ],
            max_tokens=120,
            temperature=0.7,
        )


        return response.choices[0].message.content.strip()


    except Exception as e:

        # Emotional fallback responses

        fallback_responses = {
            "happy": "I can see you're in great spirits today! That positive energy will help your crops grow beautifully.",
            "sad": "I notice you seem a bit down. Remember, every farmer has tough days, but tomorrow brings new possibilities!",
            "angry": "Take a deep breath, friend. Sometimes farming can be frustrating, but you're doing better than you think.",
            "surprised": "You look amazed! There's always something wonderful to discover in farming.",
            "fearful": "Don't worry, you're safe here. Farming can feel overwhelming at first, but you'll get the hang of it!",
            "neutral": "Nice to see you today! How can I help you with your farming journey?",
        }

        return fallback_responses.get(primary_emotion, fallback_responses["neutral"])



# Example emotion-aware scenarios


emotion_scenarios = [
    {
        "npc_name": "Wise Elder Mae",
        "detected_emotions": ["angry", "angry", "angry"],
    },
    {
        "npc_name": "Cheerful Neighbor Bob",
        "detected_emotions": ["happy", "neutral", "fearful"],
    },
    {
        "npc_name": "Gentle Librarian Luna",
        "detected_emotions": ["worried", "concerned", "sad"],
    },
]



print("💖 Emotion-Aware Dialogue System Ready!")


print("\n🎭 Example Emotional Scenarios:")


for i, scenario in enumerate(emotion_scenarios, 1):

    print(f"{i}. {scenario['npc_name']}")

    print(f"   Emotions: {' → '.join(scenario['detected_emotions'])}")

# Test emotion-aware dialogue generation
if client:
    try:
        print("💖 Testing emotion-aware dialogue generation...")

        for scenario in emotion_scenarios:
            dialogue = create_emotion_aware_dialogue(
                client,
                npc_name=scenario["npc_name"],
                detected_emotions=scenario["detected_emotions"],
            )
            print(f"{scenario['npc_name']}: {dialogue}")

        print("✅ Emotion-aware dialogue generation successful!")

    except Exception as e:
        print(f"❌ Error generating emotion-aware dialogue: {e}")

## 📊 Performance and Cost Considerations

When using AI APIs in games, it's important to consider performance and costs:

In [ ]:
# Performance and cost monitoring tools



class APIUsageTracker:
    """Track API usage for cost and performance monitoring"""


    def __init__(self):

        self.calls_made = 0

        self.total_tokens = 0


        self.response_times = []

        self.errors = 0


    def log_api_call(
        self, tokens_used: int, response_time: float, success: bool = True
    ):
        """Log an API call for tracking"""

        self.calls_made += 1
        if success:


            self.total_tokens += tokens_used

            self.response_times.append(response_time)
        else:


            self.errors += 1


    def get_stats(self) -> Dict:
        """Get usage statistics"""

        avg_response_time = (
            sum(self.response_times) / len(self.response_times)
            if self.response_times
            else 0
        )

        return {
            "total_calls": self.calls_made,
            "successful_calls": self.calls_made - self.errors,
            "total_tokens": self.total_tokens,
            "error_rate": self.errors / self.calls_made if self.calls_made > 0 else 0,
            "avg_response_time": avg_response_time,
            "estimated_cost": self.total_tokens * 0.0001,  # Example: $0.0001 per token
        }



# Best practices for game integration


best_practices = {
    "caching": "Cache AI responses to avoid repeated API calls for similar situations",
    "fallbacks": "Always have static fallback content when AI is unavailable",
    "rate_limiting": "Implement rate limiting to prevent excessive API usage",
    "async_processing": "Use async processing to avoid blocking gameplay",
    "user_consent": "Allow players to disable AI features for privacy/performance",
    "offline_mode": "Ensure game works fully without AI when internet is unavailable",
}



print("📈 API Usage Tracking System Ready!")


print("\n💡 Best Practices for Game Integration:")


for practice, description in best_practices.items():


    print(f"• {practice.replace('_', ' ').title()}: {description}")



# Example usage tracker


tracker = APIUsageTracker()


tracker.log_api_call(tokens_used=75, response_time=1.2, success=True)


tracker.log_api_call(tokens_used=0, response_time=0, success=False)



print(f"\n📊 Sample Stats: {tracker.get_stats()}")

## 🎓 Educational Summary and Next Steps

Congratulations! You've learned how to integrate advanced AI language models into game development. Let's summarize what we've accomplished and explore next steps.

### 🎯 What We've Learned

1. **API Integration**: How to authenticate and connect to UF's NaviGator API
2. **Dynamic Content Generation**: Creating contextual NPC dialogue and educational content
3. **Emotion-Aware Systems**: Adapting AI responses based on player emotional state
4. **Fallback Systems**: Ensuring game functionality when AI is unavailable
5. **Performance Monitoring**: Tracking usage and optimizing for cost and speed

### 🚀 Next Steps

1. **Implement in Game**: Add the AI manager to your PyDew Valley dialogue system
2. **Expand NPCs**: Create unique AI personalities for different characters
3. **Educational Content**: Generate dynamic farming lessons based on player progress
4. **Quest Generation**: Use AI to create unique missions and storylines
5. **Player Adaptation**: Develop systems that learn from player behavior

### 🎮 Real-World Applications

This integration demonstrates concepts used in modern game development:
- **Procedural Content Generation**: AI-created quests, dialogue, and scenarios
- **Personalized Gaming**: Adaptive experiences based on player data
- **Educational Gaming**: Context-aware teaching and guidance systems
- **Emotional Computing**: Games that respond to player emotional states

The future of gaming lies in these intelligent, adaptive experiences that provide unique value to each player!